# Machine Learning SoSe21 Practice Class

Dr. Timo Baumann, Dr. Özge Alaçam, Björn Sygo <br>
Email: baumann@informatik.uni-hamburg.de, alacam@informatik.uni-hamburg.de, 6sygo@informatik.uni-hamburg.de

## Exercise 4
**Description:** Analyse and work with a machine learning library for support vector machines <br>
**Deadline:** Saturday, 22.05.2021, 23:59 <br>
**Working together:** You can work in pairs or triples but no larger teams are allowed. <br>
&emsp;&emsp;&emsp; &emsp; &emsp; &emsp; &emsp; Please adhere to the honor code discussed in class. <br>
&emsp;&emsp;&emsp; &emsp; &emsp; &emsp; &emsp; All members of the team must get involved in understanding and coding the solution.

## Submission: 
**Christoph Brauer, Linus Geewe, Moritz Lahann**

*Also put high-level comments that should be read before looking at your code and results.*

### Goal
 1. analyze existing code of machine learning algorithms (in this case for training SVMs)
 2. experiment with integrating and using code that is available in libraries.

You will be working with two libraries, one implemented in C++ (LibSVM, see https://github.com/cjlin1/libsvm) and the other in JAVA (part of the WEKA library, see https://github.com/Waikato/weka-3.8).

Note that if you are running Windows, doing the C++ parts of this exercise in the Windows Subsystem for Linux may (or may not) simplify your life.

## Task 1: Mapping Code to Machine Learning Concepts

In the following, we ask you to relate concepts used in SVMs to the code in the two implementations. When we ask you to identify where in the code a given concept is given, please report the filename(s) and line(s) as well as the class/function/variable names.


### Kernels
1. Find the code that defines the abstract kernel definition and report what types of kernels are available in WEKA/LibSVM respectively (as well as where/how they are implemented).
2. How would you go about implementing an additional kernel?
3. Describe the process involved in computing the dot product (`svm.cpp:294ff.`, operation `Kernel::dot` for LibSVM; `weka.classifiers.functions.supportVector.CachedKernel#dotProd`, lines 292ff. for WEKA) including what are the arguments to the respective functions.
4. Describe the caching that is going on in both libraries (LRU refers to least-recently-used cache). What is being cached and why is this relevant?

### Sequential Minimal Optimization

1. As described in the lectures, SMO alternates between finding two parameters α to optimize, and optimizing their values. Find this loop in both implementations and name the methods used for performing the two steps.
2. Identify the code that performs the 'clipping' of lagrange multipliers as described in the lecture notes on page 25. Can you find the optimization computations involved in finding the new value for $α_i$ and $α_j$?
3. Describe where in the code the mapping function $ϕ$ is being computed.

### Hints
* Download both WEKA and LibSVM from the URLs given above (preferably use `git clone`).
* For LibSVM, you'll primarily look at `svm.cpp`. Any editor with syntax highlighting will do.
* Open the WEKA code in the IDE of your choice (IntelliJ, Eclipse, ...). WEKA comes with an Eclipse project definition that IntelliJ can import as well.
* Sometimes it may be easier to start the analysis with LibSVM, sometimes with WEKA -- although C++ may be less familiar, it's easy to get lost in the WEKA class hierarchies; conversely, the class hierarchy may also help understanding.

### Kernels

1. 
 * libsvm  
 The abstract kernel is defined as a class (svm.cpp:202). Specific kernels are implemented by deciding which kernel function to use in the constructor of the Kernel class (svm.cpp:253). The available kernels are LINEAR, POLY, RBF, SIGMOID, and PRECOMPUTED.

 * Weka  
 The abstract kernel class is implemented in `supportVector\Kernel.java`. The kernel implementations are classes in other files (e.g. `supportVector\RBFKernel.java`) that extend the Kernel class and overwrite its methods as needed, e.g. `buildKernel` and `evaluate`. The available kernels are Poly, NormalizedPoly, RBF, Subsequence (`StringKernel.java`), PrecomputedKernel, and Pearson universal kernel (`PUK.java`).

2.
 * libsvm  
 Define a new kernel function in the Kernel class (svm.cpp:250)  
 Add the new function to the kernel type switch case (svm.cpp:273)  
 * Weka  
 Create a new class in the supportVector folder which extends Kernel or CachedKernel.

3.
 * libsvm  
 The dot product takes two pointers as arguments. These pointers signify the start of the two vectors. The vectors are expected to be terminated with a `svm_node` with index -1. As long as the indices are equal, the product of the values of the vectors as this position is added to the total sum. The sum is then returned.

 * weka   
 The dot product takes two Instances as arguments. These are a data type that store indices and corresponding values. The implementation follows the same principle as that of libsvm, values are multiplied and summed as long as indices of the two vectors are equal.

 4.

  * libsvm
  The LRU-cache stores the sub-problems of the optimization problem, which means when sub-problems repeat the result can be quickly and efficiently read from the cache (`svm.cpp:1273,1283`).

  * weka  
  The implementation caches the evaluation results of the Kernel function. This means that results for repeating values can be read from cache and do not need to be recalculated from scratch, saving time.
 

## SMO

1. 

 * libsvm  
 The optimization loop happens at `svm.cpp:564`. `select_working_set()` is used to find the two alphas i and j to optimize. They are then updated at `svm.cpp:604` or `svm.cpp:647`.

 * weka  
 The optimization loop is located at `SMO.java:591`. This is called from the training loop `SMO.java:427` with `smo.buildClassifier`, taking a number of training examples as the argument. `examineExample` is called for an instance, which then performs one optimization step by calling `takeStep`. 

2.

 * libsvm  
 The clipping is done in the `update_alpha_status()` method at `svm.cpp:430`. The optimization computations are located as outlined in 1.

 * weka  
 The clipping happens at `SMO.java:968` or `SMO.java:987` depending on the sign of the second derivative. $a_1$ is updated to be more optimized at `SMO.java:1007` with the value of $a_2$, which is optimized at `SMO.java:965` or `SMO.java:976-986` (and then clipped). The values are saved for each step at `SMO.java:1104-1105`.

 3.




## Task 2: Working with LibSVM

In the second part, you will use the previously analysed code on the examples from the previous face classification task.

### Setup

First, you will have to make the LibSVM library available to Python.

You can use a pre-built python version for this; instructions are at

https://pypi.org/project/libsvm/

Alternatively, you can compile it from the github project you already analyzed for the first part. Further instructions are in the `README`.

To use the library, you should read the documentation in the github project's `README` to inform you how to use it. You can also view the example to get a grasp on how it works in python. It shows you the neccessary functions, but to adapt them to the tasks below, you may have to look up the parameters in the documentation.

Note: we are aware that the instructions above are sketchy. Get used to it and ask questions on the exercise forum. Once you're a machine learning practitioner, you'll get worse instructions on more badly maintained software.

### Testing different kernels for face classification

Now you have to use the feature vectors you have generated in the previous exercise submission on GDA and apply the SVM algorithm of the library to classify them. You may have to convert them to the neccessary format in a text file first to utilize the library. Try testing with different kernels, namely the linear, polynomial (degrees d = 1,2,3,4), sigmoid and radial basis kernel.

In this submission, please also report results on the full dataset, not only the small (balanced) 80-image set.

Note that you may of course use the GDA sample solution if you are unsure about your own feature extraction.

In [1]:
# GDA sample solution
import glob
import numpy as np
from PIL import Image as im

def load_imgs_numpy(path):
    imgs = []
    for index, imgpath in enumerate(glob.iglob(path + "*")):
        img = im.open(imgpath)
        imgs.append(np.array(img)[:, :, :3])
    return np.array(imgs)    

def prepare_samples(mask_images, nomask_images):
    samples = []
    for image in mask_images:
        samples.append(np.array([image, 1], dtype=object))
    for image in nomask_images:
        samples.append(np.array([image, -1], dtype=object))
    return np.array(samples)

def mean_numpy(image):
    return np.mean(image, (0, 1))

def std_numpy(image):
    return np.std(image, (0, 1))

maskFilter = [
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
[0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0],
[0, 0.5, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0],
[0, 0.5, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.5, 0.5, 0.5, 0],
[0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

def masked_mean_numpy(image, mask):
    mask = np.array([np.array(row) for row in mask])
    masked_image = image * np.stack((mask, mask, mask), 2)

    return mean_numpy(masked_image)

def feature_vector(image):
    mean = mean_numpy(image)
    masked_mean = masked_mean_numpy(image, maskFilter)
    std = std_numpy(image)
    return np.concatenate([mean, masked_mean, std])



In [13]:
# load data
mask_imgs = load_imgs_numpy("cutouts/with_mask/")
nomask_imgs = load_imgs_numpy("cutouts/without_mask/")

all_samples = prepare_samples(mask_imgs, nomask_imgs)
np.random.shuffle(all_samples)

print(all_samples.shape)

# save to file
with open('data_for_svm', 'w') as f:
    for sample in all_samples:
        target = sample[1]
        if target > 0:
            target = f"+{target}"
        else:
            target = f"{target}"

        features = feature_vector(sample[0])
        features = [f"{index + 1}:{feature}" for index, feature in enumerate(features)]

        line = " ".join([target, *features])
        f.write(line + "\n")

(1221, 2)


In [6]:
from libsvm.svmutil import *

y, x = svm_read_problem('./data_for_svm')

split_index = round(len(y) * 0.8)

y_train = y[:split_index]
y_test = y[split_index:]

x_train = x[:split_index]
x_test = x[split_index:]

m = svm_train(y_train, x_train, '-c 4')

p_label, p_acc, p_val = svm_predict(y_test, x_test, m)

Accuracy = 86.8852% (212/244) (classification)


In [10]:
kernel_types = [
    {"nr": 0, "name": "linear"},
    {"nr": 1, "name": "polynomial"},
    {"nr": 2, "name": "radial"},
    {"nr": 3, "name": "sigmoid"},
]

for kernel in kernel_types:
    nr = kernel["nr"]
    name = kernel["name"]
    if kernel["name"] == "polynomial":
        for degree in [1, 2, 3, 4]:
            
            arg = f"-t {nr} -d {degree}"
            m = svm_train(y_train, x_train, arg)

            print(f"Kernel type: {name}")
            print(f"Degrees: {degree}")
            p_label, p_acc, p_val = svm_predict(y_test, x_test, m)
            print(p_acc)

    arg = f"-t {nr}"
    m = svm_train(y_train, x_train, arg)

    print(f"Kernel type: {name}")
    p_label, p_acc, p_val = svm_predict(y_test, x_test, m)
    print(p_acc)

Kernel type: linear
Accuracy = 94.2623% (230/244) (classification)
(94.26229508196722, 0.22950819672131148, 0.5606303881517001)
Kernel type: polynomial
Degrees: 1
Accuracy = 94.6721% (231/244) (classification)
(94.67213114754098, 0.21311475409836064, 0.5841311641462237)
Kernel type: polynomial
Degrees: 2
Accuracy = 95.082% (232/244) (classification)
(95.08196721311475, 0.19672131147540983, 0.6327425826326029)
Kernel type: polynomial
Degrees: 3
Accuracy = 93.0328% (227/244) (classification)
(93.0327868852459, 0.2786885245901639, 0.49844262852347604)
Kernel type: polynomial
Degrees: 4
Accuracy = 92.623% (226/244) (classification)
(92.62295081967213, 0.29508196721311475, 0.5179958866357489)
Kernel type: polynomial
Accuracy = 93.0328% (227/244) (classification)
(93.0327868852459, 0.2786885245901639, 0.49844262852347604)
Kernel type: radial
Accuracy = 86.4754% (211/244) (classification)
(86.47540983606558, 0.5409836065573771, nan)
Kernel type: sigmoid
Accuracy = 86.4754% (211/244) (classifi

## Kernel types

Best results are obtained with polynomial kernel (2 degrees) at 95.082%, however linear kernel and poly kernel with degree 1 are within two correctly classified samples of this.

### Tune your parameters

To effectively use kernels on your task, you will have to tune the parameters $C$ and $gamma$ for your model. Try different values to improve your results. You need not search for the overall best parameters but describe and apply some systematic approach that you use.

## Hyperparameter Tuning


We are going for a gridsearch approach, which means looking at all possible combinations of the different values for $C$ and $gamma$ that we want to test. As for selecting the different values to test, we opt for a coarse search over different orders of magnetude. The idea behind this is that we want to get a rough idea of what the parameters might optimally be. Performance differences within a single order of magnetude (i.e. 0.2 vs 0.5) are typically small and subject to some randomness possibly. 

In [19]:
import itertools

# default is 1 so we cover a range of large and small orders of magnetude
c_values = [100, 10, 1, 0.1, 0.001, 0.0001]

# default is 1/number of samples (in our case 1221) so it seems absurd to go above 1
gamma_values = [1, 0.1, 0.001, 0.0001, 0.00001, 0.000001]

# 6x6 = 36 different tests
combinations = list(itertools.product(c_values, gamma_values))

accs = []

for values in combinations:
    c = values[0]
    gamma = values[1]

    # use best results from previous test
    arg = f"-t 1 -d 2 -c {c} -g {gamma}"
    m = svm_train(y_train, x_train, arg)

    print(f"Cost: {c} Gamma: {gamma}")
    p_label, p_acc, p_val = svm_predict(y_test, x_test, m)
    print(p_acc)

    accs.append(p_acc[0])

print(f"Maximum accuracy: {max(accs)}")
max_values = combinations[np.argmax(np.array(accs))]
print(f"Best combination: C {max_values[0]}, gamma {max_values[1]}")

Cost: 100 Gamma: 1
Accuracy = 94.2623% (230/244) (classification)
(94.26229508196722, 0.22950819672131148, 0.590353405313224)
Cost: 100 Gamma: 0.1
Accuracy = 94.6721% (231/244) (classification)
(94.67213114754098, 0.21311475409836064, 0.5919713092654164)
Cost: 100 Gamma: 0.001
Accuracy = 97.9508% (239/244) (classification)
(97.95081967213115, 0.08196721311475409, 0.8306561095168803)
Cost: 100 Gamma: 0.0001
Accuracy = 97.541% (238/244) (classification)
(97.54098360655738, 0.09836065573770492, 0.7968996854036183)
Cost: 100 Gamma: 1e-05
Accuracy = 91.3934% (223/244) (classification)
(91.39344262295081, 0.3442622950819672, 0.352512631702636)
Cost: 100 Gamma: 1e-06
Accuracy = 86.4754% (211/244) (classification)
(86.47540983606558, 0.5409836065573771, nan)
Cost: 10 Gamma: 1
Accuracy = 94.2623% (230/244) (classification)
(94.26229508196722, 0.22950819672131148, 0.590353405313224)
Cost: 10 Gamma: 0.1
Accuracy = 94.2623% (230/244) (classification)
(94.26229508196722, 0.22950819672131148, 0.5606

## Results

Maximum accuracy: 97.95081967213115
Best combination: C 100, gamma 0.001

We could now use that information to do a finer grid-search, for example C between 50 and 500, and gamma between 0.005 and 0.0005.

We might also test out these parameter combination with different kernel types if we had lots of time.

### Report Submission

Prepare a report of your solution as a commented Jupyter notebook (using markdown for your results and comments); include figures and results.
If you must, you can also upload a PDF document with the report annexed with your Python code.

Upload your report file to the Machine Learning Moodle Course page. Please make sure that your submission team corresponds to the team's Moodle group that you're in.